## Libraries and Dependencies

In [2]:
import torch
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.stats import norm
from pyDOE import lhs
import pandas as pd
import akshare as ak
import warnings
import datetime

np.random.seed(1234)
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
# 深度神经网络类
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        self.depth = len(layers) - 1
        self.activation = torch.nn.Tanh()
        
        layer_list = []
        for i in range(self.depth - 1):
            layer_list.append(('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1])))
            layer_list.append(('activation_%d' % i, self.activation))
        
        layer_list.append(('layer_%d' % (self.depth-1), torch.nn.Linear(layers[-2], layers[-1])))
        self.layers = torch.nn.Sequential(OrderedDict(layer_list))
        
    def forward(self, x):
        return self.layers(x)


In [4]:
# 物理信息神经网络类
class PhysicsInformedNN():
    def __init__(self, X, u, layers, lb, ub, r):
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        self.N_obs = len(u[u != 0])  # 假设u为0的为采样点
        self.X_obs = torch.tensor(X[:self.N_obs], requires_grad=True).float().to(device)
        self.u_obs = torch.tensor(u[:self.N_obs], requires_grad=False).float().to(device)
        
        self.X_samples = torch.tensor(X[self.N_obs:], requires_grad=True).float().to(device)
        
        self.r = r
        
        # Heston模型参数
        self.kappa = torch.nn.Parameter(torch.randn(1).to(device))  # 均值回归速度
        self.theta = torch.nn.Parameter(torch.randn(1).to(device))  # 长期波动率均值
        self.sigma_v = torch.nn.Parameter(torch.randn(1).to(device))  # 波动率波动率
        self.rho = torch.nn.Parameter(torch.randn(1).to(device))     # 相关性系数
        
        self.dnn = DNN(layers).to(device)
        
        # 注册参数
        self.dnn.register_parameter('kappa', self.kappa)
        self.dnn.register_parameter('theta', self.theta)
        self.dnn.register_parameter('sigma_v', self.sigma_v)
        self.dnn.register_parameter('rho', self.rho)
        
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(),
            lr=1.0,
            max_iter=50,
            max_eval=5000,
            line_search_fn="strong_wolfe"
        )
        self.iter = 0

    def net_u(self, X):
        S, v, t = X[:,0:1], X[:,1:2], X[:,2:]
        return self.dnn(torch.cat([S, v, t], dim=1))
    
    def net_f(self, X):
        S = X[:,0:1].requires_grad_(True)
        v = X[:,1:2].requires_grad_(True)
        t = X[:,2:].requires_grad_(True)
        
        u = self.net_u(torch.cat([S, v, t], dim=1))
        
        # 计算一阶导数
        u_t, = torch.autograd.grad(
            u, t, torch.ones_like(u), retain_graph=True, create_graph=True
        )
        u_S, = torch.autograd.grad(
            u, S, torch.ones_like(u), retain_graph=True, create_graph=True
        )
        u_v, = torch.autograd.grad(
            u, v, torch.ones_like(u), retain_graph=True, create_graph=True
        )
        
        # 计算二阶导数
        u_SS, = torch.autograd.grad(
            u_S, S, torch.ones_like(u_S), retain_graph=True, create_graph=True
        )
        u_Sv, = torch.autograd.grad(
            u_S, v, torch.ones_like(u_S), retain_graph=True, create_graph=True
        )
        u_vv, = torch.autograd.grad(
            u_v, v, torch.ones_like(u_v), retain_graph=True, create_graph=True
        )
        
        # 应用参数约束
        kappa = torch.exp(self.kappa)
        theta = torch.exp(self.theta)
        sigma_v = torch.exp(self.sigma_v)
        rho = torch.tanh(self.rho)
        
        # Heston PDE
        f = (
            u_t + 
            self.r*S*u_S + 
            kappa*(theta - v)*u_v + 
            0.5*v*S**2*u_SS + 
            rho*sigma_v*torch.sqrt(v)*S*u_Sv + 
            0.5*sigma_v**2*v*u_vv - 
            self.r*u
        )
        return f
    
    def loss_func(self):
        # 计算观测点误差
        u_pred = self.net_u(self.X_obs)
        loss_u = torch.mean((self.u_obs - u_pred)**2)
        
        # 计算采样点的PDE残差
        f_pred = self.net_f(self.X_samples)
        loss_f = torch.mean(f_pred**2)
        
        # 正则化项
        reg_loss = 0.001 * (self.kappa**2 + self.theta**2 + self.sigma_v**2)
        
        total_loss = loss_u + loss_f + reg_loss
        return total_loss
    
    def train(self, nIter):
        self.dnn.train()
        for i in range(nIter):
            def closure():
                self.optimizer.zero_grad()
                loss = self.loss_func()
                loss.backward()
                return loss
            
            loss_val = self.optimizer.step(closure).item()
            
            if i % 100 == 0:
                # 直接使用优化器返回的损失值，避免重复计算
                print(f"Iter {i}: Loss={loss_val:.3e}")
                print(f"kappa={torch.exp(self.kappa).item():.3f}, "
                      f"theta={torch.exp(self.theta).item():.3f}, "
                      f"sigma_v={torch.exp(self.sigma_v).item():.3f}, "
                      f"rho={torch.tanh(self.rho).item():.3f}")

    def predict(self, X):
        self.dnn.eval()
        X = torch.tensor(X).float().to(device)
        u = self.net_u(X).detach().cpu().numpy()
        f = self.net_f(X).detach().cpu().numpy()
        return u, f

In [ ]:
# 数据准备
def prepare_data():
    # 获取期权数据（示例）
    option_data = ak.option_sse_daily_sina(symbol='10006687').drop_duplicates()
    option_data['日期'] = pd.to_datetime(option_data['日期']).dt.strftime('%Y-%m-%d')
    option_data.rename(columns={'收盘': '期权收盘价'}, inplace=True)  # 假设期权收盘价列为'收盘'
    
    # 获取标的物历史数据
    stock_data = ak.fund_etf_hist_sina(symbol="sh510300")
    stock_data.columns = ['日期', '开盘', '收盘', '最高', '最低', '成交量']
    stock_data["日期"] = pd.to_datetime(stock_data["日期"])
    start_date = "2024-01-18"
    end_date = "2024-06-26"
    stock_data = stock_data[(stock_data["日期"] >= start_date) & (stock_data["日期"] <= end_date)]
    stock_data['日期'] = stock_data['日期'].dt.strftime('%Y-%m-%d')
    
    # 合并数据
    merged_data = pd.merge(option_data, stock_data[['日期', '收盘']], on='日期', how='inner')
    merged_data.rename(columns={'收盘': '股票收盘价'}, inplace=True)  # 股票收盘价列名
    
    # 添加波动率v列（假设初始值为0.5）
    merged_data['波动率v'] = 0.5  # 可以根据实际数据调整
    
    # 转换日期为数值时间戳
    merged_data['时间t'] = (pd.to_datetime(merged_data['日期']) - 
                          pd.to_datetime(start_date)).dt.days
    
    # 添加波动率v的合理范围
    v_min, v_max = 0.01, 0.5  # 波动率范围限制在合理区间
    merged_data['波动率v'] = np.clip(merged_data['波动率v'], v_min, v_max)
    
    # 特征工程
    X_market = merged_data[['股票收盘价', '波动率v', '时间t']].values
    u_market = merged_data['期权收盘价'].values  # 假设期权收盘价列名为'期权收盘价'
    
    # 添加随机采样点
    N_samples = 1000
    S_min, S_max = X_market[:,0].min(), X_market[:,0].max()
    v_min, v_max = X_market[:,1].min(), X_market[:,1].max()
    t_min, t_max = X_market[:,2].min(), X_market[:,2].max()
    
    S_samples = np.random.uniform(S_min, S_max, N_samples)
    v_samples = np.random.uniform(v_min, v_max, N_samples)
    t_samples = np.random.uniform(t_min, t_max, N_samples)
    
    X_samples = np.column_stack([S_samples, v_samples, t_samples])
    u_samples = np.zeros(N_samples)  # 采样点目标设为0
    
    # 组合数据并归一化
    X_total = np.vstack([X_market, X_samples])
    u_total = np.concatenate([u_market, u_samples])
    
    lb = X_total.min(0)
    ub = X_total.max(0)
    X_train = (X_total - lb) / (ub - lb)
    u_train = (u_total - u_total.min()) / (u_total.max() - u_total.min())
    
    return X_train, u_train, lb, ub

In [6]:
# 主程序
if __name__ == "__main__":
    # 准备数据
    X_train, u_train, lb, ub = prepare_data()
    
    # 初始化模型
    layers = [3, 30, 30, 30, 30, 1]
    r = 0.0141  # 无风险利率
    
    model = PhysicsInformedNN(X_train, u_train, layers, lb, ub, r)
    
    # 训练模型（减少迭代次数以快速测试）
    model.train(nIter=500)
    
    # 输出参数
    with torch.no_grad():
        print("\nLearned Parameters:")
        print(f"kappa = {np.exp(model.kappa.cpu().numpy()[0]):.4f}")
        print(f"theta = {np.exp(model.theta.cpu().numpy()[0]):.4f}")
        print(f"sigma_v = {np.exp(model.sigma_v.cpu().numpy()[0]):.4f}")
        print(f"rho = {np.tanh(model.rho.cpu().numpy()[0]):.4f}")

Iter 0: Loss=nan
kappa=nan, theta=nan, sigma_v=nan, rho=nan
Iter 100: Loss=nan
kappa=nan, theta=nan, sigma_v=nan, rho=nan
Iter 200: Loss=nan
kappa=nan, theta=nan, sigma_v=nan, rho=nan
Iter 300: Loss=nan
kappa=nan, theta=nan, sigma_v=nan, rho=nan
Iter 400: Loss=nan
kappa=nan, theta=nan, sigma_v=nan, rho=nan

Learned Parameters:
kappa = nan
theta = nan
sigma_v = nan
rho = nan
